# EDA

## Make data_csv and save

In [1]:
# 导入库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
root_path = './'
data_csv_raw = pd.read_csv(os.path.join(root_path, 'train.csv'))

In [3]:
def get_path(str_id):
    root_path = './'
    train_root_path = os.path.join(root_path, 'train')
    str_id_list = str_id.split('_')
    case_path = os.path.join(train_root_path, str_id_list[0])
    day_path = os.path.join(case_path, str_id_list[0] + '_' + str_id_list[1])
    slice_path_list = os.listdir(os.path.join(day_path, 'scans'))
    slice_path_10_list = []
    for item in slice_path_list:
        slice_path_10_list.append(item[:10])
    index = slice_path_10_list.index(str_id_list[2] + '_' + str_id_list[3])
    path = os.path.join(day_path, slice_path_list[index])
    return path
    
def get_day_id(str_id):
   str_id_list = str_id.split('_')
   return str_id_list[1][3:]

In [4]:
get_path('case123_day20_slice_0001')

'./train/case123/case123_day20/slice_0001_266_266_1.50_1.50.png'

In [5]:
data_csv = pd.DataFrame(columns=['id', 'path', 'segmentation_s',
                                     'segmentation_lb','segmentation_sb',  
                                     'case_id', 'day_id', 'slice_h', 
                                     'slice_w', 'px_spacing_h', 
                                     'px_spacing_w', 'slice_id'])
data_csv['id'] = data_csv_raw.id.unique()
data_csv['path'] = data_csv['id'].apply(get_path)
data_csv['case_id'] = data_csv['id'].apply(lambda x: x[4:x.index('_')])
data_csv['day_id'] = data_csv['id'].apply(get_day_id)
data_csv['slice_h'] = data_csv['path'].apply(lambda x: x[-21:-18])
data_csv['slice_w'] = data_csv['path'].apply(lambda x: x[-17:-14])
data_csv['px_spacing_h'] = data_csv['path'].apply(lambda x: x[-13:-9])
data_csv['px_spacing_w'] = data_csv['path'].apply(lambda x: x[-8:-4])
data_csv['slice_id'] = data_csv['id'].apply(lambda x: x[-4:])
# data_csv.fillna('', inplace=True)

In [6]:
data_csv_raw_ics = data_csv_raw[['id', 'class', 'segmentation']]
for item in range(int(len(data_csv_raw_ics)/3)):
    data_csv.loc[item, 'segmentation_lb'] = data_csv_raw_ics.loc[3*item, 'segmentation']
    data_csv.loc[item, 'segmentation_sb'] = data_csv_raw_ics.loc[3*item + 1, 'segmentation']
    data_csv.loc[item, 'segmentation_s'] = data_csv_raw_ics.loc[3*item + 2, 'segmentation']
# 用0替换segmentation里的nan
data_csv.fillna(0, inplace=True)

In [7]:
# 验证是否正确
data_csv.loc[data_csv['id'] == 'case123_day20_slice_0075', ['segmentation_s', 'segmentation_sb', 'segmentation_lb']]

,segmentation_s,segmentation_sb,segmentation_lb
74,12388 11 12652 14 12916 18 13180 22 13445 24 1...,0,23058 4 23323 6 23588 8 23854 9 24119 10 24385...


In [8]:
data_csv.to_csv(os.path.join('./', 'data_csv.csv'), index=False)
data_csv

,id,path,segmentation_s,segmentation_lb,segmentation_sb,case_id,day_id,slice_h,slice_w,px_spacing_h,px_spacing_w,slice_id
0,case123_day20_slice_0001,./train/case123/case123_day20/slice_0001_266_2...,0,0,0,123,20,266,266,1.50,1.50,0001
1,case123_day20_slice_0002,./train/case123/case123_day20/slice_0002_266_2...,0,0,0,123,20,266,266,1.50,1.50,0002
2,case123_day20_slice_0003,./train/case123/case123_day20/slice_0003_266_2...,0,0,0,123,20,266,266,1.50,1.50,0003
3,case123_day20_slice_0004,./train/case123/case123_day20/slice_0004_266_2...,0,0,0,123,20,266,266,1.50,1.50,0004
4,case123_day20_slice_0005,./train/case123/case123_day20/slice_0005_266_2...,0,0,0,123,20,266,266,1.50,1.50,0005
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./train/case30/case30_day0/slice_0140_266_266_...,0,0,0,30,0,266,266,1.50,1.50,0140
38492,case30_day0_slice_0141,./train/case30/case30_day0/slice_0141_266_266_...,0,0,0,30,0,266,266,1.50,1.50,0141
38493,case30_day0_slice_0142,./train/case30/case30_day0/slice_0142_266_266_...,0,0,0,30,0,266,266,1.50,1.50,0142
38494,case30_day0_slice_0143,./train/case30/case30_day0/slice_0143_266_266_...,0,0,0,30,0,266,266,1.50,1.50,0143


## Analyse for data_csv

#### Import data

In [9]:
# 导入库
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly import tools
from plotly.offline import init_notebook_mode, iplot
from plotly.subplots import make_subplots

In [10]:
data_csv = pd.read_csv('./data_csv.csv')
data_csv

,id,path,segmentation_s,segmentation_lb,segmentation_sb,case_id,day_id,slice_h,slice_w,px_spacing_h,px_spacing_w,slice_id
0,case123_day20_slice_0001,./train/case123/case123_day20/slice_0001_266_2...,0,0,0,123,20,266,266,1.5,1.5,1
1,case123_day20_slice_0002,./train/case123/case123_day20/slice_0002_266_2...,0,0,0,123,20,266,266,1.5,1.5,2
2,case123_day20_slice_0003,./train/case123/case123_day20/slice_0003_266_2...,0,0,0,123,20,266,266,1.5,1.5,3
3,case123_day20_slice_0004,./train/case123/case123_day20/slice_0004_266_2...,0,0,0,123,20,266,266,1.5,1.5,4
4,case123_day20_slice_0005,./train/case123/case123_day20/slice_0005_266_2...,0,0,0,123,20,266,266,1.5,1.5,5
...,...,...,...,...,...,...,...,...,...,...,...,...
38491,case30_day0_slice_0140,./train/case30/case30_day0/slice_0140_266_266_...,0,0,0,30,0,266,266,1.5,1.5,140
38492,case30_day0_slice_0141,./train/case30/case30_day0/slice_0141_266_266_...,0,0,0,30,0,266,266,1.5,1.5,141
38493,case30_day0_slice_0142,./train/case30/case30_day0/slice_0142_266_266_...,0,0,0,30,0,266,266,1.5,1.5,142
38494,case30_day0_slice_0143,./train/case30/case30_day0/slice_0143_266_266_...,0,0,0,30,0,266,266,1.5,1.5,143


#### Number of pictures each case_id

In [11]:
# case数量图
case = data_csv['case_id'].value_counts()
case = case.sort_index()
# trace1 = go.Scatter(
#     x=list(case.index),
#     y=list(case.values),
#     name = 'slice_num',
#     mode = 'markers + lines'
# )
# layout = go.Layout(title='number of pictures each case_id',
#                    xaxis= dict(title= 'case_id',ticklen= 5,zeroline= False),
#                    yaxis= dict(title= 'number_of_pictures',ticklen= 5,zeroline= False))
# fig = go.Figure(data=[trace1], layout=layout)
# fig.show()

In [12]:
case_distribution = case.value_counts()
case_distribution_index = pd.Series(case_distribution.index).apply(lambda x: str(x) + ' pictures each case')
fig = {
  "data": [
    {
      "values": case_distribution.values,
      "labels": case_distribution_index,
      "domain": {"x": [0, .5]},
      "name": "Number Of",
      "hoverinfo":"label+percent",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"number_of_pictures_each_case_distribution",
                'height':600, 
                "width":800
    }
}
iplot(fig)

#### Number of day each case_id

In [13]:
case_index = pd.Series(case.index)
case_day = case_index.apply(lambda x: len(data_csv.loc[data_csv['case_id'] == x, 'day_id'].unique()))
case_day.index = case.index
# trace2 = go.Scatter(
#     x=list(case_day.index),
#     y=list(case_day.values),
#     name = 'slice_num',
#     mode = 'markers + lines'
# )
# layout = go.Layout(title='number of day each case_id',
#                    xaxis= dict(title= 'case_id',ticklen= 5,zeroline= False),
#                    yaxis= dict(title= 'number_day',ticklen= 5,zeroline= False))
# fig = go.Figure(data=[trace2], layout=layout)
# fig.show()

In [14]:
case_day_distribution = case_day.value_counts()
case_day_distribution_index = pd.Series(case_day_distribution.index).apply(lambda x: str(x) + ' days each case')
fig = {
  "data": [
    {
      "values": case_day_distribution.values,
      "labels": case_day_distribution_index,
      "domain": {"x": [0, .5]},
      "name": "Number Of",
      "hoverinfo":"label+percent",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"number_of_days_each_case_distribution",
                'height':600, 
                "width":800
    }
}
iplot(fig)

#### Number of pictures each day distribution

In [15]:
slice_of_day = []
for item_case_id in case.index:
    case_csv = data_csv.loc[data_csv['case_id'] == item_case_id, :]
    day_of_case = case_csv['day_id'].value_counts().values
    slice_of_day.extend(day_of_case)
slice_of_day_csv = pd.Series(slice_of_day)
day_num = slice_of_day_csv.value_counts()
day_num_index = pd.Series(day_num.index).apply(lambda x: str(x) + ' pictures each day')
fig = {
  "data": [
    {
      "values": day_num.values,
      "labels": day_num_index,
      "domain": {"x": [0, .5]},
      "name": "Number Of",
      "hoverinfo":"label+percent",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"number_of_pictures_each_day_distribution",
        'height':500, 
        "width":700
    }
}
iplot(fig)

1. 题目里说测试集由全新的case和部分已有case里抽走的部分图片组成
2. 有没有可能原本每天都要扫描144张，这5.47%的80张就是被抽到测试集里的
3. case118_day0 就只有80张

#### Area of segmentation distribution

In [16]:
def get_area(x):
    if x == 0:
        return 0
    else:
        list_int = []
        list_str = x.split()
        for item in list_str:
            list_int.append(int(item))
        return sum(list_int[1::2])

In [55]:
# 不同case或者同一case不同天数的某个slice_id的s、lb、sb的面积分布
def show_area_of_slice_id(slice_id=1):
    mask_slice_id_id_analyse = mask_csv.loc[mask_csv['slice_id'] == slice_id, :]
    trace__1 =go.Box(y = mask_slice_id_id_analyse['area_s'],
                       name = "area of s",
                       marker = dict(color = 'rgba(255, 128, 255, 0.8)'),
                       text= mask_slice_id_id_analyse['case_id'].apply(lambda x: 'case' + str(x)))
    trace1__2 =go.Box(y = mask_slice_id_id_analyse['area_lb'],
                       name = "area of lb",
                       marker = dict(color = 'rgba(255, 128, 2, 0.8)'),
                       text= mask_slice_id_id_analyse['case_id'].apply(lambda x: 'case' + str(x)))
    trace__3 =go.Box(y = mask_slice_id_id_analyse['area_sb'],
                       name = "area of sb",
                       marker = dict(color = 'rgba(0, 255, 200, 0.8)'),
                       text= mask_slice_id_id_analyse['case_id'].apply(lambda x: 'case' + str(x)))
    layout = dict(title = 'Area of segmentation distribution on slice_id: {}'.format(slice_id),
                  yaxis= dict(title= 'Area',ticklen= 5,zeroline= False),
                  height=700, width=900)
    fig = dict(data = [trace__1, trace1__2, trace__3], layout = layout)
    iplot(fig)

In [88]:
# 每个case的s、lb、sb的面积随着splice_id的变化曲线
def show_area_of_case(c_id=123):
    if len(mask_csv.loc[mask_csv['case_id'] == c_id, :]) != 0:
        case_id = c_id
    else:
        case_id = mask_csv['case_id'].unique()[0]
    day_id_for_case_id =  mask_csv.loc[mask_csv['case_id'] == case_id, 'day_id'].unique()
    day_id_for_case_id = np.sort(day_id_for_case_id)
    
    # layout = dict(title = 'case: ' + str(case_id), 
    #               xaxis= dict(title= 'slice_id',ticklen= 5,zeroline= False), 
    #               yaxis= dict(title= 'area',ticklen= 5,zeroline= False))
        
    fig = make_subplots(rows=3, cols=len(day_id_for_case_id))
    
    for item in range(len(day_id_for_case_id)):
        mask_csv_for_analyse = mask_csv.loc[(mask_csv['case_id'] == case_id) & (mask_csv['day_id'] == day_id_for_case_id[item]), :]
        fig.append_trace(go.Scatter(
                        x = mask_csv_for_analyse['slice_id'],   
                        y = mask_csv_for_analyse['area_s'],    
                        mode = "lines",
                        name = 'day_{}_s'.format(day_id_for_case_id[item])),
                        row=item+1, col=1)
        fig.append_trace(go.Scatter(
                        x = mask_csv_for_analyse['slice_id'],   
                        y = mask_csv_for_analyse['area_sb'],    
                        mode = "lines",
                        name = 'day_{}_sb'.format(day_id_for_case_id[item])),
                        row=item+1, col=2)
        fig.append_trace(go.Scatter(
                        x = mask_csv_for_analyse['slice_id'],   
                        y = mask_csv_for_analyse['area_lb'],    
                        mode = "lines",
                        name = 'day_{}_lb'.format(day_id_for_case_id[item])),
                        row=item+1, col=3)
        # data = [trace_1, trace_2, trace_3]
        #     ) 
    fig.update_layout(height=700, width=900,
                      title_text="Area_of_segmentation_for_slice_id_of_case_{}".format(case_id))
    fig.show()

In [19]:
mask_csv = pd.DataFrame(columns=['case_id', "px_spacing_h", 'px_spacing_w',
                                 'slice_id', 'day_id', 'path',
                                 'area_s', 'area_lb','area_sb'])
mask_csv[['case_id', "px_spacing_h", 
          'px_spacing_w', 'slice_id', 
          'day_id', 'path']] = data_csv[['case_id', "px_spacing_h", 
                                 'px_spacing_w', 'slice_id', 'day_id', 'path']]
mask_csv['area_s'] = data_csv['segmentation_s'].apply(get_area)
mask_csv['area_lb'] = data_csv['segmentation_lb'].apply(get_area)
mask_csv['area_sb'] = data_csv['segmentation_sb'].apply(get_area)
# 标准化，乘以每个像素的大小
for item in range(len(mask_csv)):
    mask_csv.loc[item, 'area_s'] = mask_csv.loc[item, 'area_s'] * (mask_csv.loc[item, 'px_spacing_h'] * mask_csv.loc[item, 'px_spacing_w'])
    mask_csv.loc[item, 'area_lb'] = mask_csv.loc[item, 'area_lb'] * (mask_csv.loc[item, 'px_spacing_h'] * mask_csv.loc[item, 'px_spacing_w'])
    mask_csv.loc[item, 'area_sb'] = mask_csv.loc[item, 'area_sb'] * (mask_csv.loc[item, 'px_spacing_h'] * mask_csv.loc[item, 'px_spacing_w'])

In [90]:
show_area_of_case(118)

1. x轴是slice_id，y轴是stomach，large_bowel，small_bowel区域面积
2. 面积的计算方式是stomach，large_bowel，small_bowel区域像素总数乘以该图片每个像素的大小（pixel spacing）
3. 看起来变化还算连续

In [56]:
show_area_of_slice_id(60)

1. 这是某个slice_id下不同case以及同一个case不同天数下的stomach，large_bowel，small_bowel面积分布，基本上没啥规律。。。。

#### Relative position transformation of splice_id

In [21]:
def get_x_new(x_raw):
    if x_raw != "0":
        x_raw = x_raw.split()
        for item in range(len(x_raw)):
            x_raw[item] = int(x_raw[item])
        x_raw_1 = x_raw[::2]
        x_raw_2 = x_raw[1::2]
        x_new = []
        for item in range(len(x_raw_1)):
            for item_index in range(x_raw_2[item]):
                x_new.append(x_raw_1[item] + item_index)
    else:
        x_new = [0]
    return x_new

In [22]:
# 距离用的题目里的 Dice coefficient 
def get_distance(x1_raw, x2_raw):
    x1_new = get_x_new(x1_raw)
    x2_new = get_x_new(x2_raw)
    x1_new_and_x2_new = list(set(x1_new) & set(x2_new))
    if x1_new == [0] and x2_new == [0]:
        return 1
    else:
        return 2 * len(x1_new_and_x2_new) / (len(x1_new) + len(x2_new))

In [84]:
def show_dice_coefficient_of_case(c_id=123):
    if len(data_csv.loc[data_csv['case_id'] == c_id, :]) != 0:
        case_id = c_id
    else:
        case_id = data_csv['case_id'].unique()[0]
    day_id_for_case_id =  data_csv.loc[data_csv['case_id'] == case_id, 'day_id'].unique()
    day_id_for_case_id = np.sort(day_id_for_case_id)
    
    fig = make_subplots(rows=3, cols=len(day_id_for_case_id))
    
    for item in range(len(day_id_for_case_id)):
        data_csv_for_analyse = data_csv.loc[(data_csv['case_id'] == case_id) & (data_csv['day_id'] == day_id_for_case_id[item]), :]
        data_csv_for_analyse.index = range(len(data_csv_for_analyse))
        distance_s = []
        distance_lb = []
        distance_sb = []
        for item_splice_id in range(1, len(data_csv_for_analyse)):
            distance_s.append(get_distance(data_csv_for_analyse.loc[item_splice_id-1, 'segmentation_s'], data_csv_for_analyse.loc[item_splice_id, 'segmentation_s']))
            distance_lb.append(get_distance(data_csv_for_analyse.loc[item_splice_id-1, 'segmentation_lb'], data_csv_for_analyse.loc[item_splice_id, 'segmentation_lb']))
            distance_sb.append(get_distance(data_csv_for_analyse.loc[item_splice_id-1, 'segmentation_sb'], data_csv_for_analyse.loc[item_splice_id, 'segmentation_sb']))
        
        fig.append_trace(go.Scatter(
                        x = list(range(len(distance_s))),   
                        y = distance_s,    
                        mode = "lines",
                        name = 'day_{}_s'.format(day_id_for_case_id[item])),
                        row=item+1, col=1)
        fig.append_trace(go.Scatter(
                        x = list(range(len(distance_sb))),   
                        y = distance_sb,      
                        mode = "lines",
                        name = 'day_{}_sb'.format(day_id_for_case_id[item])),
                        row=item+1, col=2)
        fig.append_trace(go.Scatter(
                        x = list(range(len(distance_lb))),   
                        y = distance_lb,      
                        mode = "lines",
                        name = 'day_{}_lb'.format(day_id_for_case_id[item])),
                        row=item+1, col=3)
    fig.update_layout(height=700, width=900,
                      title_text="Dice_coefficient_of_segmentation_for_slice_id_of_case_{}".format(case_id))
    fig.show()    

In [85]:
show_dice_coefficient_of_case()

1. 某个case下某一天的stomach，large_bowel，small_bowel区域在随着slice_id变化时前一张与后一张的位置dice_coefficient度量变化（越接近1表示前一张与这一张变化不大）
2. 不同行代表不同天，不同列是stomach，large_bowel，small_bowel
3. 图像上看起来仅仅是在stomach，large_bowel，small_bowel开始出现和忽然消失的那一刻变化较大，其余时刻变化不大，加上前面的面积变化也比较连续
4. 能不能先识别出测试集里的图片属于哪个case的哪一天？然后用

In [87]:
show_dice_coefficient_of_case(118)

1. case118的day0只有80张，但不是拿走的都是s、sb、lb区域消失以后的图片，不影响
2. 其余80张的不知道什么情况

## Analyse for pictures